# Scripts for preparing metatdata for PDBBind-Opt

In this script, we will create CSV files containing relevant metadata that will be used for the PDBBind-Opt workflow to process structures.

In [1]:
import pandas as pd
from tqdm import tqdm
import re
import requests
import math

from rdkit import Chem

In [2]:
def get_smiles_from_rcsb(comp_id: str):
    """
    Query ligand SMILES from RCSB

    Parameters
    ----------
    comp_id: str
        The ligand ID, usually a three-letter code
    
    Returns
    -------
    smi: str
        The SMILES of the query ligand. If fail to get, will return a vacant string
    """
    query = '''{chem_comp(comp_id: "%s") {
        rcsb_chem_comp_descriptor {
        SMILES_stereo SMILES InChI
        }
    }
    }''' % comp_id
    query = re.sub(r'\s+', ' ', query)
    try:
        res = requests.get('https://data.rcsb.org/graphql?query=' + query)
        smi = res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['SMILES_stereo']
        if smi is None:
            smi = res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['SMILES']
        if smi is None:
            m = Chem.MolFromInchi(res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['InChI'])
            smi = Chem.MolToSmiles(m)
        assert smi is not None, "No reference smiles"
        return smi
    except:
        return ""


def regularize_binding_data(typ, sign, number, unit):
    
    # handle number that have uncertainty
    if '+-' in number:
        number = number.split('+-')[0]
    number = float(number)
    # handle sign
    sign = sign[1] + sign[0] if sign in ['=>', '=<'] else sign
    # convert Ka/Kb to Kd
    typ = typ.lower()
    if typ == 'ka' or typ == 'kb':
        typ = 'kd'
        assert unit.endswith('^-1'), f'Incorrect unit for Ka/Kb: {unit}'
        unit = unit.rstrip('^-1')
        number = 1 / number
    
    if unit == 'M':
        lognum = math.log10(number)
    elif unit == 'mM':
        lognum = math.log10(number) - 3
    elif unit == 'uM':
        lognum = math.log10(number) - 6
    elif unit == 'nM':
        lognum = math.log10(number) - 9
    elif unit == 'pM':
        lognum = math.log10(number) - 12
    elif unit == 'fM':
        lognum = math.log10(number) - 15
    else:
        lognum = None

    return {
        "measurement": typ,
        "sign": sign,
        "value": number,
        "unit": unit,
        "logvalue": lognum
    }

## Parse Original PDBBind Data

In [3]:
def parse_pdbbind_metadata(index='../raw/index/INDEX_general_PL.2020'):
    data = []
    with open(index) as f:
        for line in f:
            if line.startswith('#'):
                continue
            if line:
                content = line.strip().split()
                if not content[6].endswith(')'):
                    ligand = content[6][1:]
                else:
                    ligand = content[6][1:-1]
                    
                data.append({
                    "PDBID": content[0],
                    "Resolution": content[1],
                    "Year": content[2],
                    "Binding Affinity": content[3],
                    "Ligand": ligand.lstrip('_'),
                    "Note": ' '.join(content[7:])
                })
    data = pd.DataFrame(data)
    return data

pdbbind_data = parse_pdbbind_metadata('../raw_data_pdbbind/index/INDEX_general_PL.2020').set_index("PDBID").sort_index()
pdbbind_ids = pdbbind_data.index.unique()
print("Number of data in PDBBind v2020:", len(pdbbind_ids))
pdbbind_data

Number of data in PDBBind v2020: 19443


,Resolution,Year,Binding Affinity,Ligand,Note
PDBID,,,,,
10gs,2.20,1998,Ki=0.4uM,VWW,
11gs,2.30,1999,Ki=1.5uM,2-mer,redundant to 3gss
13gs,1.90,1999,Ki=24uM,SAS,
16pk,1.60,1998,Ki=6uM,BIS,X-ray(Trypanosoma brucei PGK) while Ki(yeast PGK)
184l,1.80,1995,Kd=19uM,I4B,ligand is compound 39
...,...,...,...,...,...
966c,1.90,1999,Ki=23nM,RS2,
9abp,1.97,1992,Kd=0.01uM,GLA/GLB,isomer
9hvp,2.80,1992,Ki=4.5nM,0E9,


## Parse BioLiP

In [ ]:
%%bash
wget https://zhanggroup.org/BioLiP/download/BioLiP.txt.gz
gunzip BioLiP.txt.gz

In [4]:
columns = [
    'PDBID',
    'Receptor chain',
    'Resolution',
    'Binding site',
    'Ligand CCD',
    'Ligand chain',
    'Ligand serial number',
    'Binding site residues',
    'Binding site residues renumbered',
    'Catalytic site residues',
    'Catalytic site residues renumbered',
    'EC number',
    'GO terms',
    'Binding affinity (manual)',
    'Binding affinity (Binding MOAD)',
    'Binding affinity (PDBbind-CN)',
    'Binding affinity (Binding DB)',
    'UniProt ID',
    'PubMed ID',
    'Ligand residue sequence number',
    'Receptor sequence'
]
raw_df = pd.read_csv('BioLiP.txt', sep='\t', names=columns, low_memory=False, keep_default_na=False, na_values=[None, ""])
raw_df

,PDBID,Receptor chain,Resolution,Binding site,Ligand CCD,Ligand chain,Ligand serial number,Binding site residues,Binding site residues renumbered,Catalytic site residues,...,EC number,GO terms,Binding affinity (manual),Binding affinity (Binding MOAD),Binding affinity (PDBbind-CN),Binding affinity (Binding DB),UniProt ID,PubMed ID,Ligand residue sequence number,Receptor sequence
0,101m,A,2.07,BS01,HEM,A,1,F43 R45 V68 S92 H93 H97 I99 Y103,F44 R46 V69 S93 H94 H98 I100 Y104,NaN,...,"1.11.1.-,1.7.-.-","0004601,0005344,0005737,0015671,0016491,001652...",NaN,NaN,NaN,NaN,P02185,NaN,155,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,102m,A,1.84,BS01,HEM,A,1,F43 R45 T67 L89 S92 H93 H97 I99 Y103,F44 R46 T68 L90 S93 H94 H98 I100 Y104,NaN,...,"1.11.1.-,1.7.-.-","0004601,0005344,0005737,0015671,0016491,001652...",NaN,NaN,NaN,NaN,P02185,NaN,155,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
2,103m,A,2.07,BS01,HEM,A,1,F43 R45 S92 H93 H97 I99 Y103,F44 R46 S93 H94 H98 I100 Y104,NaN,...,"1.11.1.-,1.7.-.-","0004601,0005344,0005737,0015671,0016491,001652...",NaN,NaN,NaN,NaN,P02185,NaN,155,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
3,104m,A,1.71,BS01,HEM,A,1,F43 R45 V68 S92 H93 H97 I99 Y103 F138,F43 R45 V68 S92 H93 H97 I99 Y103 F138,NaN,...,"1.11.1.-,1.7.-.-","0004601,0005344,0005737,0015671,0016491,001652...",NaN,NaN,NaN,NaN,P02185,NaN,155,VLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRF...
4,105m,A,2.02,BS01,HEM,A,1,F43 R45 H64 V68 L89 H93 H97 I99,F43 R45 H64 V68 L89 H93 H97 I99,NaN,...,"1.11.1.-,1.7.-.-","0004601,0005344,0005737,0015671,0016491,001652...",NaN,NaN,NaN,NaN,P02185,NaN,155,VLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRF...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933384,9xim,C,2.40,BS02,MN,C,1,E181 E217 D245 D292,E178 E214 D242 D289,H54 D57 M88 E181 K183 E217 H220 D245 D255 D257...,...,5.3.1.5,"0000287,0005737,0005975,0009045,0016853,004273...",NaN,NaN,NaN,NaN,P12851,1610791.0,395,QATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIGAY...
933385,9xim,C,2.40,BS03,MN,C,2,E217 H220 D255 D257,E214 H217 D252 D254,H54 D57 M88 E181 K183 E217 H220 D245 D255 D257...,...,5.3.1.5,"0000287,0005737,0005975,0009045,0016853,004273...",NaN,NaN,NaN,NaN,P12851,1610791.0,396,QATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIGAY...
933386,9xim,D,2.40,BS01,XLS,D,1,H54 W137 E181 K183 H220 D292,H52 W135 E179 K181 H218 D290,H54 D57 M88 E181 K183 E217 H220 D245 D255 D257...,...,5.3.1.5,"0000287,0005737,0005975,0009045,0016853,004273...",NaN,NaN,NaN,NaN,P12851,1610791.0,397,VQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIGA...
933387,9xim,D,2.40,BS02,MN,D,1,E181 E217 D245 D292,E179 E215 D243 D290,H54 D57 M88 E181 K183 E217 H220 D245 D255 D257...,...,5.3.1.5,"0000287,0005737,0005975,0009045,0016853,004273...",NaN,NaN,NaN,NaN,P12851,1610791.0,395,VQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIGA...


## Prepare PDBBind-Opt

In [18]:
biolip_pdbbind = raw_df.query('PDBID in @pdbbind_ids')
biolip_pdbbind_dict = {pdbid: subdf for pdbid, subdf in biolip_pdbbind.groupby("PDBID")}

datas = {
    "sm": [],
    "poly": []
}
patt = re.compile(r"([a-zA-Z50]+)([~<>=]+)([\d.eE+-]+)([^\s,]+)")
for pdbid, row in tqdm(list(pdbbind_data.iterrows())):
    # determine if small molecule or polymers
    ligand_ccd = row['Ligand']
    if bool(re.search(r'[^a-zA-Z0-9]', ligand_ccd)):
        category = 'poly'
    else:
        category = 'sm'


    # parse binding data
    binding_string = str(row['Binding Affinity'])
    binding_data = regularize_binding_data(*tuple(re.findall(patt, binding_string))[0])
    binding_data['source'] = 'PDBBind'
    binding_data['origin'] = binding_string

    if category == 'poly' or (pdbid not in biolip_pdbbind_dict):
        record = {"PDBID": pdbid, "Ligand CCD": ligand_ccd, 'Ligand chain': None, 'Ligand residue sequence number': None}
        record.update(binding_data)
        datas[category].append(record)
    else:
        biolip_record = biolip_pdbbind_dict[pdbid]
        cnt = 0
        for chain, subdf in biolip_record.groupby('Ligand chain'):
            for _, row in subdf.iterrows():
                if row['Ligand CCD'] == ligand_ccd:
                    record = {
                        "PDBID": pdbid, "Ligand CCD": ligand_ccd, 
                        'Ligand chain': row['Ligand chain'], 
                        'Ligand residue sequence number': row['Ligand residue sequence number'].replace(' ', '')
                    }
                    record.update(binding_data)
                    datas[category].append(record)
                    cnt += 1
                    break
        if cnt == 0:
            record = {"PDBID": pdbid, "Ligand CCD": ligand_ccd, 'Ligand chain': None, 'Ligand residue sequence number': None}
            record.update(binding_data)
            datas[category].append(record)
    

100%|██████████| 19443/19443 [00:10<00:00, 1934.40it/s]


In [19]:
for category in datas:
    datas[category] = pd.DataFrame(datas[category])
    print(f"PDBBind-{category} #PDBID: {datas[category]['PDBID'].unique().shape[0]}")
    datas[category].to_csv(f'PDBBind_{category}.csv', index=None)

PDBBind-sm #PDBID: 16738
PDBBind-poly #PDBID: 2705
